# Family Dependencies

This notebook allocates births to their representative parents based on the historical shares of the total number of newborns by the age of their mothers. The notebook adjusts the age of the mother in a way that factors in the historical age difference between fathers and mothers, so that adults are representative of both genders. The notebook also computes a measure of fertility rates by the age of representative adults.

The data on the median age at first marriage of men and women are obtained from the U.S. Census Bureau at www.census.gov/hhes/families/data/marital.html (downloaded on August 8, 2016). The publication is called "Estimated Median Age at First Marriage, by Sex: 1890 to the Present". These data cover the period 1890 to 2015. Yearly estimates are available starting in 1947; prior to that year, only decennial estimates corresponding to census years are available.

The data on the number of births by age of mother is taken from the Centers for Disease Control and Prevention's National Center for Health Statistics (NCHS). The data can be found under "Cohort Fertility Tables" at www.cdc.gov/nchs/cohort_fertility_tables.htm. We downloaded the data on August 8, 2016. We use two sets of historical tables. The first set is put together by Hamilton and Cosgrove (2010) and consists of two separate data releases. The first release is titled "Table 1. Cental birth rates, by live-birth order, current age, and race of women in each cohort from 1911 through 1991: United States, 1960-2005". The second release is titled "Table 1. Cental birth rates, by live-birth order, current age, and race of women in each cohort from 1957 through 1995: United States, 2006-2009". The methodology and data sources are consistent across the two releases. We saved the data in separate CSV files that are read in the notebook below. Consistent with the timing of population estimates in the Census, women who belong to, say, the 1960 cohort, are born within a twelve-month period centered on January 1, 1960, and thus running from July 1, 1959, to June 30, 1960.

The second set of data is based on the historical publication "Fertility Tables for Birth Cohorts by Color: United States, 1917-73", DHEW Publication No. (HRA) 76-1152, U.S. Department of Health, Education, and Welfare, April 1976, Washington D.C. The link to a PDF of that publication is http://www.cdc.gov.nchs/dad/misc/fertiltbacc.pdf (last accessed on August 8, 2016). We extracted the information from Table 4A, "Central birth rates for all women during each year 1917-1973 by age and life-birth order foreach cohort from 1868 to 1959, Total birth group", and entered that information manually into a CSV file.

The two sets of files do not use identical methodologies. In particular, there are methodological differences in judgmental adjustments for underrestration of births, underenumeration of the number of women, and misstatement of age for all years. That said, the differences appear too small to have a material effect on the key simulation results in our paper. We also note that the shares of births by age of the mother are fairly similar for the overlapping period between 1960 and 1973.

The allocation of babies to their parents of a given age is smoothed because the average age difference between men and women is fractional. This smoothing avoids discrete jumps in the number of babies that might otherwise occur if the average age difference rounded to a different period. For yearly data, if the average age difference is, say, 2.7 years, then the age of the representative adult associated with a 14-year-old mother would be $(14+2.7/2) = 15.35$ years. In that case, we would linearly allocate babies to the neighboring yearly bins---with 65 percent of babies assigned to parents aged 16 years and 35 percent of babies assigned to parents aged 17 years---so as to be consistent with the average.

### Estimating the age difference between male and female parents

In [ ]:
# Reading the marriage age data
using DelimitedFiles
periods = Array{Int64,1}(readdlm("RawData/Median_age_first_marriage.csv",',',skipstart=1)[:,1])
age_marriage = readdlm("RawData/Median_age_first_marriage.csv",',',skipstart=1)[:,2:3]

# Extending the data through 2100 by carrying the average difference in recent years
using Statistics
periods = [periods ; ((periods[1,1]+1):2100)]
age_marriage = [age_marriage; [mean(age_marriage[1:5,1])*ones(2100-periods[1,1],1)  mean(age_marriage[1:5,2])*ones(2100-periods[1,1],1)]]
age_marriage = [age_marriage age_marriage[:,1]-age_marriage[:,2]]
recentdiff = age_marriage[end,end]

# Sorting the data
p = sortperm(periods)
periods=periods[p]
age_marriage=age_marriage[p,:]

# Fitting a fifth-order polynomial...
n = length(periods)
X = [ones(n,1) (periods.+0.5) (periods.+0.5).^2 (periods.+0.5).^3 (periods.+0.5).^4 (periods.+0.5).^5]
Y = Array{Float64,1}(age_marriage[:,3])
b = inv(X'*X)*X'*Y

# ...and evaluating the polynomial at all periods of interest
perY = 1900.5:2100
perQ = 1900.125:.25:2100
Fitted_age_diff_Y = [ones(length(perY),1) perY perY.^2 perY.^3 perY.^4 perY.^5]*b;
Fitted_age_diff_Q = [ones(length(perQ),1) perQ perQ.^2 perQ.^3 perQ.^4 perQ.^5]*b;

# Forcing the polynomial approximation to converge to the average difference in recent years
AgeDiff_w_Y = [zeros(115,1); (1:(length(perY)-115))./(length(perY)-115)]
AgeDiff_w_Q = [zeros(460,1); (1:(length(perQ)-460))./(length(perQ)-460)]
MarriageAgeDiff_Y = (1.0.-AgeDiff_w_Y).*Fitted_age_diff_Y + AgeDiff_w_Y*recentdiff
MarriageAgeDiff_Q = (1.0.-AgeDiff_w_Q).*Fitted_age_diff_Q + AgeDiff_w_Q*recentdiff

# Exporting data to CSV files
writedlm("CleanData/marriageAgeDiff_Y.csv", MarriageAgeDiff_Y, header = true, ',')
writedlm("CleanData/marriageAgeDiff_Q.csv", MarriageAgeDiff_Q, header = true, ',')

### Historical estimates of the share of live births by age of mothers

In [ ]:
# Reading and re-arranging the 1960-2009 data
fertility_1960_2009 = readdlm("RawData/Births_by_age_of_mothers_1960_2009.csv",',',skipstart=6)[:,[1:2;4]]
years_fertile = minimum(fertility_1960_2009[:,2]):maximum(fertility_1960_2009[:,2]);

In [ ]:
# Create matrix whose first column is calendar year and subsequent columns 
# have number of births by years_fertile
fertility_AC_1960_2009 = NaN*ones(length(1960:2009),length(years_fertile)+1)
fertility_AC_1960_2009[:,1] = 1960:2009
years_1960_2009=1960:2009
for ii = 1:size(fertility_1960_2009,1)
    fertility_AC_1960_2009[Int64(fertility_1960_2009[ii,1])-1959,Int64(fertility_1960_2009[ii,2])-Int64(years_fertile[1])+2] = fertility_1960_2009[ii,3]
end

In [ ]:
# Reading and re-arranging the 1960-2009 data
fertility_1917_1973 = readdlm("RawData/Births_by_age_of_mothers_1917_1973.csv",',',skipstart=1)
fertility_AC_1917_1973 = NaN*ones(1973-1917+1,length(years_fertile)+1)
fertility_AC_1917_1973[:,1] = 1917:1973
for ii = 1:size(fertility_1917_1973,1)
    for jj = 3:size(fertility_1917_1973,2)
        if fertility_1917_1973[ii,jj]>=0.0
            fertility_AC_1917_1973[ii+jj-38,jj-1] = fertility_1917_1973[ii,jj]
        end
    end
end

### Merging the two datasets

In [ ]:
# Stacking the share of babies by age of mothers: 1900:2099
fertility_AC_1917_2009 = [fertility_AC_1917_1973[1:end-14,:]; fertility_AC_1960_2009]
births_1917_2009 = sum(fertility_AC_1917_2009[:,2:end],dims=2)
share_births_1900_2100_Y = [fertility_AC_1917_2009[:,1] fertility_AC_1917_2009[:,2:end] ./ (births_1917_2009*ones(1,length(years_fertile)))]
share_births_1900_2100_Y = [ones(17,1)*share_births_1900_2100_Y[1:1,:] ; share_births_1900_2100_Y ; ones(90,1)*share_births_1900_2100_Y[end:end,:]]
share_births_1900_2100_Y[1:17,1] = 1900:1916
share_births_1900_2100_Y[111:end,1] = 2010:2099;

In [ ]:
# Interpolating the share of births to a quarterly period frequency...
include("ordernorep.jl")
include("spline_cubic.jl")
share_births_1900_2100_perQ_ageY = NaN*ones(length(perQ),length(years_fertile))
for aa=1:length(years_fertile) 
    share_births_1900_2100_perQ_ageY[:,aa]=spline_cubic(share_births_1900_2100_Y[:,1].+0.5,share_births_1900_2100_Y[:,aa+1],perQ)
end

# and interpolating the periods to a quarterly age frequency
ageQ = 14.125:0.25:50
share_births_1900_2100_perQ_ageQ = NaN*ones(length(perQ),length(ageQ))
for qq=1:length(perQ) 
    share_births_1900_2100_perQ_ageQ[qq,:]=spline_cubic(years_fertile.+0.5,share_births_1900_2100_perQ_ageY[qq,:],ageQ,1,1)
    share_births_1900_2100_perQ_ageQ[qq,share_births_1900_2100_perQ_ageQ[qq,:].<0.0].=0.0
end

# Rescaling shares to ensure they sum to 1
share_births_1900_2100_perQ_ageQ = share_births_1900_2100_perQ_ageQ./(sum(share_births_1900_2100_perQ_ageQ,dims=2)*ones(1,length(ageQ)));

In [ ]:
# Writing the shares of births per age of mother to CSV files
writedlm("CleanData/share_births_mothers_Y.csv",share_births_1900_2100_Y[:,2:end]', header = false, ',')
writedlm("CleanData/share_births_mothers_Q.csv",share_births_1900_2100_perQ_ageQ', header = false, ',')
writedlm("CleanData/age_mother_Y.csv",years_fertile.+0.5, header = false, ',')
writedlm("CleanData/age_mother_Q.csv",ageQ, header = false, ',')

In [ ]:
# Changing permissions
run(`chmod 664 CleanData/marriageAgeDiff_Y.csv`);
run(`chmod 664 CleanData/marriageAgeDiff_Q.csv`);
run(`chmod 664 CleanData/share_births_mothers_Y.csv`);
run(`chmod 664 CleanData/share_births_mothers_Q.csv`);
run(`chmod 664 CleanData/age_mother_Y.csv`);
run(`chmod 664 CleanData/age_mother_Q.csv`);